In [ ]:
# %load_ext autoreload
# %autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['legend.fontsize'] = 24
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['figure.titlesize'] = 30
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA2.09.48'
lat=yu.LatticeEnsemble(ens)

In [ ]:
cfgList=[]
filesDic={}
files2ptKey={}
files3ptKey={}

data={}

def loadData(basePath):
    f={}
    for key in filesDic.keys():
        f[key]={}
        for file in filesDic[key]:
            f[key][file] = h5py.File(basePath + file + '.h5','r')

    for key in files2ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in files3ptKey:
        if key not in data.keys():
            data[key]={}
        for file in filesDic[key]:
            data[key][file]={}
            tF=f[key][file]
            for projAStr in tF.keys():
                projA=eval(projAStr)
                data[key][file][projA]={}
                for projBStr in tF[projAStr].keys():
                    projB=eval(projBStr)
                    data[key][file][projA][projB]={}
                    for projg in tF[projAStr][projBStr].keys():
                        data[key][file][projA][projB][projg]={}
                        for cont in tF[projAStr][projBStr][projg][cfgList[0]].keys():
                            data[key][file][projA][projB][projg][cont]=np.array([
                                f[key][file][projAStr][projBStr][projg][cfg][cont][()] for cfg in cfgList
                                ])

    for key in filesDic.keys():
        for file in filesDic[key]:
            f[key][file].close()

path_projData='/p/project/pines/li47/code/projectData/'

basePath_NJN=path_projData+'nucleon_sigma_term/cA2.09.48/NJN/proj/'
basePath_NJNpi=path_projData+'nucleon_sigma_term/cA2.09.48/NJNpi_GEVP/proj/'
basePath_loop='/p/project/pines/li47/code/scratch/run/'+'nucleon_sigma_term/cA2.09.48/QuarkLoops_pi0_insertion/data_post_merged/'

with h5py.File(basePath_NJN+'NJN.h5') as f:
    cfgList_NJN=list(yu.deepKey(f,3).keys())

with h5py.File(basePath_NJNpi+'N_2pt.h5') as f:
    cfgList_NJNpi=list(yu.deepKey(f,3).keys())

# with h5py.File(basePath_NJNpi+'N_2pt-j_old.h5') as f:
#     cfgList_temp=list(yu.deepKey(f,3).keys())


cfgList=list( set(cfgList_NJN) & set(cfgList_NJNpi))
# cfgList=cfgList_NJNpi

cfgList.sort()
Ncfg=len(cfgList)
print(Ncfg)


filesDic={}
files2ptKey=[]
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['NJN']
filesDic['NJNpi']=['NJN-pi0i']
loadData(basePath_NJN)

filesDic={}
files2ptKey=['NN','NNpi','NpiN','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T','N_2pt-pi0i']
filesDic['NpiN']=['D1ii','N_2pt-pi0f']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt','N_2pt-pi0f-pi0i','T-pi0f','D1ii-pi0i']
files3ptKey=['NJN','NJNpi']
filesDic['NJN']=['N_2pt-j']
filesDic['NJNpi']=['B','W','Z','N_2pt-j-pi0i','N_2pt-j&pi0i','T-j']
loadData(basePath_NJNpi)

# filesDic={}
# files2ptKey=['NN','NNpi','NpiN','NpiNpi']
# filesDic['NN']=['N_2pt']
# filesDic['NNpi']=['T']
# filesDic['NpiN']=['D1ii']
# filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
# files3ptKey=['NJNpi']
# filesDic['NJNpi']=['B','W','Z']
# loadData(basePath_NJNpi)

data['others']={}
with h5py.File(basePath_loop+'insertLoop.h5') as f:
    data['others']['u+d_id_vev']=np.array([f[cfg]['u+d_id_vev'][()] for cfg in cfgList])
    data['others']['u-d_g5_vev']=np.array([f[cfg]['u-d_g5_vev'][()] for cfg in cfgList])
with h5py.File(basePath_loop+'pi0Loop.h5') as f:
    data['others']['pi0Loop_g5_vev']=np.array([f[cfg]['pi0Loop_g5_vev'][()] for cfg in cfgList])

# getDat

In [ ]:
test_NNpi=1; test_NpiNpi=1; test_NJN=1; test_NJNpi=1; noD1ii=False

diags_subtract={'N_2pt-j_50'}

diags_key={}
diags_all=set(); diags_pi0Loop=set(); diags_jLoop=set()
for key in data.keys():
    if key in ['others']:
        continue
    diags_key[key] = set(data[key])
    diags_all = diags_all | diags_key[key] - diags_subtract
    for diag in data[key]:
        t=diag.split('-')
        if 'pi0i' in t or 'pi0f' in t:
            diags_pi0Loop.add(diag)
        if 'j' in t:
            diags_jLoop.add(diag)

diags_pi0Loop = diags_pi0Loop - diags_subtract
diags_jLoop = diags_jLoop - diags_subtract

diags_loopful = diags_pi0Loop | diags_jLoop
diags_loopless = diags_all - diags_loopful
diags_jLoopless = diags_all - diags_jLoop
diags_all_2pt = diags_key['NN'] | diags_key['NNpi'] | diags_key['NpiNpi']
diags_all_3pt = diags_all - diags_all_2pt

projgs_P012=['P0','P01','P02']
projgs_Pz12=['Pz','P01','P02']

Ntime=yu.deepKey(data['NN'],5).shape[1]

def getDat2pt(opa,opb,projg,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat2pt(opa1,opb,projg,diags)-np.sqrt(1/3)*getDat2pt(opa2,opb,projg,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat2pt(opa,opb1,projg,diags)-np.sqrt(1/3)*getDat2pt(opa,opb2,projg,diags)

    if tpa in ['p'] and tpb in ['p']:
        key='NN'
        return data[key]['N_2pt'][proja][projb][projg][tpa+'_'+tpb]+np.zeros([Ncfg,Ntime])
    elif tpa in ['p']:
        key='NNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['T']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0i']:
                return [tpa+'_p'] if tpb =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])
    elif tpb in ['p']:
        key='NpiN'
        t_diags = diags & diags_key[key]
        if noD1ii and 'D1ii' in t_diags:
            t=np.conj(getDat2pt(opb,opa,projg,{'T'}))
            t_diags = t_diags - {'D1ii'}
        else:
            t=0
        def get_conts(diag):
            if diag in ['D1ii']:
                return [tpa+'_'+tpb]
            if diag in ['N_2pt-pi0f']:
                return ['p_'+tpb] if tpa =='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return np.conj(test_NNpi)*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])+t
    else:
        key='NpiNpi'
        t_diags = diags & diags_key[key]
        if noD1ii and 'D1ii-pi0i' in t_diags:
            t=np.conj(getDat2pt(opb,opa,projg,{'T-pi0f'}))
            t_diags = t_diags - {'D1ii-pi0i'}
        else:
            t=0
        def get_conts(diag):
            if diag in ['B_2pt','W_2pt','Z_2pt']:
                return [tpa+'_'+tpb]
            if diag in ['M_correct_2pt']:
                return [tpa+'_'+tpa] if tpa==tpb else []
            if diag in ['N_2pt-pi0f-pi0i']:
                return ['p_p'] if tpa==tpb=='p&pi0' else []
            if diag in ['T-pi0f']:
                return ['p_'+tpb] if tpa=='p&pi0' else []
            if diag in ['D1ii-pi0i']:
                return [tpa+'_p'] if tpb=='p&pi0' else []
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NpiNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]] for dc in dcs],axis=0)+np.zeros([Ncfg,Ntime])+t

def getDat2ptMat(ops,projg,diags):
    t=np.transpose([[getDat2pt(opa,opb,projg,diags) for opb in ops] for opa in ops],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(opa,opb,Gc,iso,projg,tf,diags):
    (tpa,proja)=opa; (tpb,projb)=opb

    if tpa in ['12']:
        opa1=('n&pi+',proja); opa2=('p&pi0',proja)
        return np.sqrt(2/3)*getDat3pt(opa1,opb,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa2,opb,Gc,iso,projg,tf,diags)
    if tpb in ['12']:
        opb1=('n&pi+',projb); opb2=('p&pi0',projb)
        return np.sqrt(2/3)*getDat3pt(opa,opb1,Gc,iso,projg,tf,diags)-np.sqrt(1/3)*getDat3pt(opa,opb2,Gc,iso,projg,tf,diags)

    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]

    if tpa in ['p'] and tpb in ['p']:
        key='NJN'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['NJN','N_2pt-j','N_2pt-j_50']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont) for diag in t_diags for cont in get_conts(diag)]
        return test_NJN*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])       
    elif tpa in ['p']:
        key='NJNpi'
        t_diags = diags & diags_key[key]
        def get_conts(diag):
            if diag in ['B','W','Z']:
                return [tpa+'_j'+iso+'_'+tpb+'_deltat_'+str(tf)]
            if diag in ['N_2pt-j&pi0i','N_2pt-j-pi0i','NJN-pi0i']:
                return [tpa+'_p'+'_j'+iso+'_deltat_'+str(tf)] if tpb == 'p&pi0' else []
            if diag in ['T-j']:
                return [tpa+'_'+tpb+'_j'+iso+'_deltat_'+str(tf)]
        dcs = [(diag,cont)  for diag in t_diags for cont in get_conts(diag)]
        return test_NJNpi*np.sum([data[key][dc[0]][proja][projb][projg][dc[1]][:,1:tf,i_Gc] for dc in dcs],axis=0) + np.zeros([Ncfg,tf-1])
    elif tpb in ['p']:
        if projb[0] != (0,0,0):
            return np.zeros([Ncfg,tf-1])
        return np.conj(getDat3pt(opb,opa,Gc,iso,projg,tf,diags)[:,::-1]) * (1 if Gc in ['id','gt'] else (-1))
    else:
        return np.zeros([Ncfg,tf-1])
    
def getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags):
    t=np.transpose([[getDat3pt(opa,opb,Gc,iso,projg,tf,diags) for opb in opbs] for opa in opas],[2,3,0,1])
    return t

tfList=[10,12,14]
def getDat(opas,opbs,Gc,iso,projg,diags):
    pta=opas[0][1][0]; ptb=opbs[0][1][0]
    assert(pta==(0,0,0)); pc=ptb
    if pc==(0,0,0) and projg=='Pzt':
        projg='Pz'
    if projg in ['Pz','Pzt']:
        projg_2pt='P0'
    else:
        projg_2pt=projg
    dat2ptA=getDat2ptMat(opas,projg_2pt,diags)
    dat2ptB=getDat2ptMat(opbs,projg_2pt,diags)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(opas,opbs,Gc,iso,projg,tf,diags)
    if (Gc,iso) == ('id','+'):
        dat_jLoop=data['others']['u+d_id_vev']
    elif (Gc,iso) == ('g5','-'):
        dat_jLoop=data['others']['u-d_g5_vev']
    else:
        dat_jLoop=np.zeros(Ncfg)

    t=np.array([
        np.array([dat2ptA[cfg],dat2ptB[cfg]]+[np.array([dat3pt[tf][cfg] for tf in tfList],dtype=object)]+[dat_jLoop[cfg]],dtype=object)
    for cfg in range(Ncfg)], dtype=object)
    return t


pt0=(0,0,0); pt1=(0,0,1); pt2=(0,0,-1); pt3=(1,0,0)
tp0='p'; tp1='n&pi+'; tp2='p&pi0'; tp3='12'
op00=(tp0,(pt0,'a')); op01=(tp1,(pt0,('N1\\pi1','a'))); op02=(tp2,(pt0,('N1\\pi1','a'))); op03=(tp3,(pt0,('N1\\pi1','a')))

op10=(tp0,(pt1,'a')); op11=(tp1,(pt1,('N1\\pi0','a'))); op12=(tp2,(pt1,('N1\\pi0','a'))); op13=(tp3,(pt1,('N1\\pi0','a')))
op101=(tp0,(pt1,'a')); op111=(tp1,(pt1,('N0\\pi1','a'))); op121=(tp2,(pt1,('N0\\pi1','a'))); op131=(tp3,(pt1,('N0\\pi1','a')))
op102=(tp0,(pt1,'a')); op112=(tp1,(pt1,('N2\\pi1','a'))); op122=(tp2,(pt1,('N2\\pi1','a'))); op132=(tp3,(pt1,('N2\\pi1','a')))
op103=(tp0,(pt1,'a')); op113=(tp1,(pt1,('N2\\pi1','b'))); op123=(tp2,(pt1,('N2\\pi1','b'))); op133=(tp3,(pt1,('N2\\pi1','b')))

op20=(tp0,(pt2,'a')); op21=(tp1,(pt2,('N1\\pi0','a'))); op22=(tp2,(pt2,('N1\\pi0','a'))); op23=(tp3,(pt2,('N1\\pi0','a')))
op201=(tp0,(pt2,'a')); op211=(tp1,(pt2,('N0\\pi1','a'))); op221=(tp2,(pt2,('N0\\pi1','a'))); op231=(tp3,(pt2,('N0\\pi1','a')))

op30=(tp0,(pt3,'a')); op31=(tp1,(pt3,('N1\\pi0','a'))); op32=(tp2,(pt3,('N1\\pi0','a'))); op33=(tp3,(pt2,('N1\\pi0','a')))
op301=(tp0,(pt3,'a')); op311=(tp1,(pt3,('N0\\pi1','a'))); op321=(tp2,(pt3,('N0\\pi1','a'))); op331=(tp3,(pt2,('N0\\pi1','a')))

# 2pt

In [ ]:
def run2pt(opa,opb,diags=diags_all,title=None,ylim1=None,ylim2=[.4,1.1],preFactor=1,axs=None,avgConjFlag=False,xshift=0,projgs=['P01','P0','P02'],color=None,fmt=None,label=None):
    tmin=1; tmax=16

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=opa[0]+'_'+opb[0]+(': '+title if title!=None else '')
        fig.suptitle(title)

    for projg in projgs:
        colorDic={'P01':'r','P02':'b','P0':'g'}
        if color == None:
            color=colorDic[projg]
        
        dat=preFactor*getDat2pt(opa,opb,projg,diags)
        if avgConjFlag:
            dat=preFactor*(getDat2pt(opa,opb,projg,diags)+np.conj(getDat2pt(opb,opa,projg,diags)))/2

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.real(t)
            return t
        (mean,cov,err) = yu.jackknife(dat,tFunc)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # print(tMean,tErr)
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt=fmt,elinewidth=2,capsize=6,mfc='white',label=label)
        axs[irow,icol].set_xticks(range(0,tmax,1))  
        if ylim1=='sym':
            t=max(abs(np.array(axs[irow,icol].get_ylim())))
            axs[irow,icol].set_ylim([-t,t])
        else:
            axs[irow,icol].set_ylim(ylim1)
        

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
        #     t=np.real(t)
        #     t=np.log(t/np.roll(t,-1,axis=0))
        #     return t
        # (mean,cov,err) = yu.jackknife(dat,tFunc)
        # tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        # axs[irow,icol].errorbar(np.arange(tmin,tmax)+xshift,tMean,tErr,color=color,fmt='s',elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(0,tmax,1))
        # axs[irow,icol].set_ylim(ylim2)
        # if opa[1][0] == (0,0,0):
        #     axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        # elif np.linalg.norm(opa[1][0]) == 1.0:
        #     axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
        #     axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
        #     axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange')        

In [ ]:
(opa,opb)=(op00,op00)
run2pt(opa,opb,color='r',fmt='s',projgs=['P0'])

In [ ]:
(opx,opy,opz)=(op00,op01,op02)

# N-Npi: test isospin 
projgs=['P0']
fig,axs=yu.getFigAxs(1,1,8,10); #fig.suptitle('imag')
(opa,opb)=(opx,opy)
run2pt(opa,opb,preFactor=-1j,axs=axs,projgs=projgs,color='r',fmt='s',xshift=-0.3,label=r'$p\to n\,\pi^+$')
(opa,opb)=(opx,opz)
run2pt(opa,opb,diags=diags_loopless,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=-0.,avgConjFlag=True,projgs=projgs,color='g',fmt='d',label=r'$p\to p\,\pi^0$: conn')
run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=0.3,avgConjFlag=True,projgs=projgs,color='b',fmt='o',label=r'$p\to p\,\pi^0$: full') # ylim1=[-1e-10,0]

axs[0,0].set_xlabel(r'$t_{sink}$')
axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend();

In [ ]:
(opx,opy,opz)=(op00,op01,op02)

projgs=['P0']; diags=diags_loopless
fig,axs=yu.getFigAxs(1,1,8,10); axs[0,0].set_ylim([0,0.001e-7]); #fig.suptitle('title')
(opa,opb)=(opy,opy)
run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=-0.3,color='r',fmt='s',label=r'$n\,\pi^+\to n\,\pi^+$',projgs=projgs)
(opa,opb)=(opy,opz)
run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.,color='g',fmt='d',label=r'$n\,\pi^+\to p\,\pi^0$',projgs=projgs)
(opa,opb)=(opz,opz)
run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=0.3,color='b',fmt='o',label=r'$p\,\pi^0\to p\,\pi^0$',projgs=projgs)
axs[0,0].set_xlabel(r'$t_{sink}$')
axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend()

projgs=['P0']; diags=diags_all
fig,axs=yu.getFigAxs(1,1,8,10); axs[0,0].set_ylim([0,0.001e-7]); #fig.suptitle('title')
(opa,opb)=(opy,opy)
run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=-0.3,color='r',fmt='s',label=r'$n\,\pi^+\to n\,\pi^+$',projgs=projgs)
(opa,opb)=(opy,opz)
run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.,color='g',fmt='d',label=r'$n\,\pi^+\to p\,\pi^0$',projgs=projgs)
(opa,opb)=(opz,opz)
run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=0.3,color='b',fmt='o',label=r'$p\,\pi^0\to p\,\pi^0$',projgs=projgs)
axs[0,0].set_xlabel(r'$t_{sink}$')
# axs[0,0].set_ylabel(r'correlation functions')
axs[0,0].legend()

In [ ]:
(opx,opy,opz)=(op00,op01,op02)
# (opx,opy,opz)=(op10,op11,op12)
# (opx,opy,opz)=(op10,op111,op121)
# (opx,opy,opz)=(op20,op21,op22)
# (opx,opy,opz)=(op30,op31,op32)

# N-N
# (opa,opb)=(opx,opx)
# run2pt(opa,opb)

# N-Npi  vs. Npi-N
# # noD1ii=True
# (opa,opb)=(opx,opy)
# run2pt(opa,opb,preFactor=1,title='real',ylim1='sym')
# run2pt(opb,opa,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,preFactor=-1j,title='imag')
# run2pt(opb,opa,preFactor=-1j,title='imag')
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=1,title='real',ylim1='sym')
# run2pt(opb,opa,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,preFactor=-1j,title='imag')
# run2pt(opb,opa,preFactor=-1j,title='imag')
# noD1ii=False

# N-Npi: real imag, loop
# diags=diags_all
# (opa,opb)=(opx,opy)
# run2pt(opa,opb,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,preFactor=-1j,title='imag')
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=1,title='real',ylim1='sym')
# run2pt(opa,opb,preFactor=-1j,title='imag',avgConjFlag=False)
# run2pt(opa,opb,preFactor=-1j,title='imag',avgConjFlag=True)
# diags=diags_loopless
# run2pt(opa,opb,diags=diags,preFactor=1,title='real, loopless')
# run2pt(opa,opb,diags=diags,preFactor=-1j,title='imag, loopless')
# diags=diags_loopful
# run2pt(opa,opb,diags=diags,preFactor=1,title='real, loopful')
# run2pt(opa,opb,diags=diags,preFactor=-1j,title='imag, loopful')

# N-Npi: avg N-pi0i and N-pi0f
# diags={'N_2pt-pi0i','N_2pt-pi0f'}
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,diags=diags,preFactor=-1j)
# run2pt(opb,opa,diags=diags,preFactor=-1j)
# run2pt(opa,opb,diags=diags,preFactor=-1j,avgConjFlag=True)

# N-Npi: test isospin 
# fig,axs=yu.getFigAxs(1,2,8,10); fig.suptitle('real')
# (opa,opb)=(opx,opy)
# run2pt(opa,opb,preFactor=1,axs=axs)
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=1*(-1*np.sqrt(2)),axs=axs,xshift=0.5,avgConjFlag=True,ylim1='sym')
# fig,axs=yu.getFigAxs(1,2,8,10); fig.suptitle('imag')
# (opa,opb)=(opx,opy)
# run2pt(opa,opb,preFactor=-1j,axs=axs)
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=0.5,avgConjFlag=True) # ylim1=[-1e-10,0]
# diags=diags_loopless
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,diags=diags,preFactor=-1j*(-1*np.sqrt(2)),axs=axs,xshift=-0.5,avgConjFlag=True)

# strange negative correlation
# (opx,opy,opz)=(op10,op11,op12)
# diags={'N_2pt-pi0i','N_2pt-pi0f'}
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),xshift=0.5,avgConjFlag=False,projgs=['P0'],diags=diags)
# (opa,opb)=(opz,opx)
# run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),xshift=0.5,avgConjFlag=False,projgs=['P0'],diags=diags)
# (opa,opb)=(opx,opz)
# run2pt(opa,opb,preFactor=-1j*(-1*np.sqrt(2)),xshift=0.5,avgConjFlag=True,projgs=['P0'],diags=diags)

In [ ]:
# tests for Npi-Npi
(opx,opy,opz)=(op00,op01,op02)
(opx,opy,opz)=(op10,op11,op12)
# (opx,opy,opz)=(op10,op111,op121)
# (opx,opy,opz)=(op20,op21,op22)
# (opx,opy,opz)=(op30,op31,op32)

# BWZ vs M vs loops
# # (opa,opb)=(opy,opy)
# (opa,opb)=(opz,opz)
# # (opa,opb)=(opy,opz)
# run2pt(opa,opb,title='all')
# run2pt(opa,opb,diags={'B_2pt','W_2pt','Z_2pt'},title='BWZ')
# run2pt(opa,opb,diags={'M_correct_2pt'},title='M')
# run2pt(opa,opb,diags={'T-pi0f','D1ii-pi0i'},title='1Loop')
# run2pt(opa,opb,diags={'N_2pt-pi0f-pi0i'},title='2Loops')

# n&pi+_p&pi0 vs p&pi0_n&pi+
# diags={'W_2pt'}; preFactor=1
# # (opa,opb)=(opy,opy)
# (opa,opb)=(opy,opz)
# # (opa,opb)=(opy,opz)
# run2pt(opa,opb,title='all',diags=diags,preFactor=preFactor)
# run2pt(opb,opa,title='all',diags=diags,preFactor=preFactor)

# isospin
# diags=diags_all
# fig,axs=yu.getFigAxs(1,2,8,10); axs[0,0].set_ylim([0,0.001e-7])
# (opa,opb)=(opy,opy)
# run2pt(opa,opb,diags,preFactor=1,axs=axs,xshift=0)
# (opa,opb)=(opz,opz)
# run2pt(opa,opb,diags,preFactor=2,axs=axs,xshift=-0.5)
# (opa,opb)=(opy,opz)
# run2pt(opa,opb,diags,preFactor=(-1*np.sqrt(2)),axs=axs,xshift=0.5)

# 2pt GEVP

In [ ]:

def run2ptGEVP(ops,diags=diags_all,title=None,ylim1=[.3,1.1],ylim2=[1e-4,1.5],axs=None,isospinCheck=False,xshift=0,projgs=['P01']):
    N_op=len(ops); tRef=1
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        # title=', '.join([op[0] for op in ops]) + (': '+title if title!=None else '')
        fig.suptitle(title)

    # yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])


    for projg in projgs:
        xshift_internal=0 if len(projgs)==1 else {'P0':0,'P01':-0.3,'P02':0.3}[projg]
        xshift_internal+=xshift
        dat=getDat2ptMat(ops,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
        fmtDic={0:'s',1:'d',2:'o'}
        for i in range(N_op):
            ttmax=tmax-i*3
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt=fmtDic[i],color=colorDic[i], label='1')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim1)
        
        # axs[irow,icol].legend([r'$E_{N}$',r'$E_{N\pi}$'])

        if ops[0][1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='red',label='1')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='green',label='1',linestyle='dashed')

            handles, _ = axs[irow,icol].get_legend_handles_labels()
            labels = [r'$N(\vec{0})$',r'$N(\vec{1})\pi(\vec{1})$',r'$E_{N}$',r'$E_{N\pi}$']
            order = [0,1,2,3]
            axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],ncol=2)

        elif np.linalg.norm(ops[0][1][0]) == 1.0:
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='red',label='1')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='green',label='1',linestyle='dashed') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='blue',label='1',linestyle='dashdot') 
            # axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            # axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange') 

            handles, _ = axs[irow,icol].get_legend_handles_labels()
            labels = [r'$N(\vec{1})$',r'$N(\vec{1})\pi(\vec{0})$',r'$N(\vec{0})\pi(\vec{1})$',r'$E_{N}$',r'$E_{N\pi;1}$',r'$E_{N\pi;2}$']
            order = [0,1,2,3,4,5]
            axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],ncol=2)


        axs[irow,icol].set_xlabel(r'$t_{sink}$')
        axs[irow,icol].set_ylabel(r'effective energies')

        icol=1
        for j in range(1):
            if isospinCheck and j!=0:
                continue
        
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
                # return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
            (mean,cov,err)=yu.jackknife(dat,tFunc)

            colorDic={0:(1,0,0),1:(1,0.3,0.3),2:(1,0.5,0.5),3:'purple',4:'orange',5:'cyan',6:'pink'}
            fmtDic={0:'s',1:'<',2:'>'}
            for i in range(N_op):
                tmin=2
                ttmax=tmax
                tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
                print(j,i)
                print(np.transpose([tMean,tErr]))

                if isospinCheck:
                    t={'p':1,'n&pi+':np.sqrt(2/3),'p&pi0':np.sqrt(1/3)}[ops[i][0]]
                    tMean/=t; tErr/=t
                    
                # temp_min=8
                # t_factor=1 if i!=1 else 1/np.sqrt(2)
                # print(i)
                # print(yu.npRound(tMean[temp_min:]*t_factor,3))
                # print(yu.npRound(tErr[temp_min:]*t_factor,3))

                axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt=fmtDic[i],color=colorDic[i],
                label='1'.format(i+1))
            axs[irow,icol].set_xticks(range(0,tmax,1))
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])
            if ops[0][1][0] == (0,0,0):
                handles, _ = axs[irow,icol].get_legend_handles_labels()
                labels = [r'$v^N_{N}\equiv1$',r'$v^N_{N\pi}$']
                order = [0,1]
                axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order])
            else:
                handles, _ = axs[irow,icol].get_legend_handles_labels()
                labels = [r'$v^N_{N}\equiv1$',r'$v^N_{N\pi;1}$',r'$v^N_{N\pi;2}$']
                order = [0,1,2]
                axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],loc=(0.1,0.52))

        axs[irow,icol].set_xlabel(r'$t_{sink}$')
        axs[irow,icol].set_ylabel(r'eigenvectors')

        return fig

In [ ]:
ops=[op00,op03]
fig=run2ptGEVP(ops)

with PdfPages('/p/project/pines/li47/code/forDownload/cA48_p000.pdf') as pdf:
    pdf.savefig(fig)

ops=[op10,op13,op131]
run2ptGEVP(ops)

with PdfPages('/p/project/pines/li47/code/forDownload/cA48_p001.pdf') as pdf:
    pdf.savefig(fig)


In [ ]:
(opx,opy,opz,opn)=(op00,op01,op02,op03)
(opx,opy,opz,opn)=(op10,op11,op12,op13)
(opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

projgs=['P01']
projgs=projgs_P012

# p with different ops
# ops=[opx,opy]
# run2ptGEVP(ops,projgs=projgs)
# ops=[opx,opz]
# run2ptGEVP(ops,projgs=projgs)
# ops=[opx,opy,opz]
# run2ptGEVP(ops,projgs=projgs)
# # run2ptGEVP(ops,isospinCheck=True,projgs=projgs)
# ops=[opx,opn]
# run2ptGEVP(ops,projgs=projgs)

# D1ii test
# ops=[opx,opz]
# noD1ii=True
# run2ptGEVP(ops)
# noD1ii=False
# run2ptGEVP(ops)

In [ ]:
# fig, axs=yu.getFigAxs(1,1,Lrow=8,Lcol=10)

# diags=diags_all
# projg='P01'

# noD1ii=True
# ops=[op10,op121]
# ops=[op30,op321]
# dat=getDat2ptMat(ops,projg,diags)
# noD1ii=False

# # for tf0 in range(1,10):
# for tf0 in range(1,6):
#     def tFunc(dat):
#         t=np.mean(dat,axis=0)
#         (eVals,eVecs)=yu.GEVP(t,tf0)
#         t=eVecs[:,:,0].T
#         t=t/t[0,:]
#         return np.imag(t[1,:])
#     (mean,cov,err)=yu.jackknife(dat,tFunc)
    
#     tmin=tf0+1; tmax=16
#     tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
#     axs[0,0].errorbar(np.arange(tmin,tmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=tf0)
# axs[0,0].legend()
# axs[0,0].set_ylim([-0.1,0.1])
# None

# 3pt

In [ ]:
def run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None,color='r',xshift=0):
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+(': '+title if title!=None else ''))

    # yu.addColHeader(axs,['3pt'])

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        x_shift += xshift
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*flip*(t[2][i_tf][:,0,0])) for i_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(i_tf,tMean,tErr)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color=color,elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(-6,6,1))
        axs[irow,icol].set_ylim(ylim1)

        # irow=0; icol=1
        # def tFunc(dat):
        #     t=np.mean(dat,axis=0)
            
        #     def getRes(i_tf):
        #         tf=tfList[i_tf]
        #         nom=t[2][i_tf][:,0,0]
        #         denom=np.sqrt( np.real(t[0][tf]) * np.real(t[1][tf]) *\
        #         np.real(t[0][1:tf])[::-1]/np.real(t[0][1:tf]) *\
        #         np.real(t[1][1:tf]/np.real(t[1][1:tf])[::-1]) )[:,0,0]
        #         return np.real(preFactor * (nom/denom-vevFactor*t[3]) )
        #     t=np.array([getRes(i_tf) for i_tf in range(len(tfList))],dtype=object)
        #     return t

        # (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        # colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        # fmtDic={10:'s',12:'d',14:'o'}

        # for i_tf in range(len(tfList)):
        #     tf=tfList[i_tf]
        #     tMean=mean[i_tf];tErr=err[i_tf]
        #     axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
        #         fmt=fmtDic[tf],color=colorDic[(0,0)],elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(-6,6,2))
        # axs[irow,icol].set_ylim(ylim2)


In [ ]:
(opx,opy,opz,opn)=(op00,op01,op02,op03)
# (opx,opy,opz,opn)=(op10,op11,op12,op13)
# (opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

ratio=0.15*1j*10
ratio=1j

(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
# (Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
# (Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],1)
(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
# (Gc,iso,projgs,preFactor)=('id','+',['P0'],1)

diags=diags_jLoopless
# diags={'B'}

fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
# (opa,opb)=(op01,op01)
# run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,axs=axs)
# axs[0,0].set_ylim([1.5e-9,1.68e-9])

(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,axs=axs,color='r')

(opa,opb)=(op00,opy)
run3pt(opa,opb,Gc,iso,projgs,ratio*preFactor,diags,axs=axs,color='g',xshift=0.2)
# (opa,opb)=(op00,opz)
# run3pt(opa,opb,Gc,iso,projgs,ratio*preFactor*(-np.sqrt(2)),diags,axs=axs,color='b')



In [ ]:
(opx,opy,opz,opn)=(op00,op01,op02,op03)
# (opx,opy,opz,opn)=(op10,op11,op12,op13)
(opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)

diags=diags_jLoopless # | {'N_2pt-j'}

fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,opz)
diags={'N_2pt-j&pi0i'}
run3pt(opa,opb,Gc,iso,projgs,1j,diags,axs=axs,color='r')
diags={'N_2pt-j-pi0i'}
run3pt(opa,opb,Gc,iso,projgs,1j,diags,axs=axs,color='b')


In [ ]:
def run3pt(opa,opb,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None,color='r',xshift=0,mfc=None,label=None):
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+(': '+title if title!=None else ''))

    # yu.addColHeader(axs,['3pt'])

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        x_shift += xshift
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        dat=getDat([opa],[opb],Gc,iso,projg,diags)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*flip*(t[2][i_tf][:,0,0])) for i_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            # print(i_tf,tMean,tErr)
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc=mfc,label=label)
        axs[irow,icol].set_xticks(range(-6,7,1))
        axs[irow,icol].set_ylim(ylim1)
        # axs[irow,icol].set_yscale('log')

mpl.rcParams['legend.fontsize'] = 30

(opx,opy,opz,opn)=(op00,op01,op02,op03)
# (opx,opy,opz,opn)=(op10,op11,op12,op13)
(opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)

diags=diags_jLoopless # | {'N_2pt-j'}

fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,opz)
diags={'N_2pt-j&pi0i'}
run3pt(opa,opb,Gc,iso,projgs,1j,diags,axs=axs,mfc='white',label='1')
diags={'B','W','Z','NJN-pi0i'}
run3pt(opa,opb,Gc,iso,projgs,1j,diags,axs=axs,label='1')
irow=0;icol=0
handles, _ = axs[irow,icol].get_legend_handles_labels()
labels = [r'$t_{\sink}=10$: M only',r'$t_{\sink}=12$:',r'$t_{\sink}=14$:',r'$t_{\sink}=10$: others',r'$t_{\sink}=12$:',r'$t_{\sink}=14$:']
order = [0,1,2,3,4,5]
axs[irow,icol].legend([handles[idx] for idx in order],[labels[idx] for idx in order],ncol=1,fontsize=35)
axs[irow,0].set_xlabel(r'$t_{ins}-t_{sink}/2$')
axs[irow,0].set_ylabel(r'$<0|O_N\,J\,\bar{O}_{N\pi}|0>$')
;

# 3pt2ptGEVP

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=4; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        # fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+'; '+', '.join([op[0] for op in opas]) + '; '+', '.join([op[0] for op in opbs]) + (': '+title if title!=None else ''))
        fig.suptitle(title)
    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[1],tRef)

            # [[] for j in [0,1]], for i in [0,1]]

            # print(t[0].shape)

            def get_eVec(eVecs):
                t=eVecs[:,0,0]
                t=eVecs[:,:,0]/t[:,None]
                return np.mean(t[tfAvgMin:tfAvgMax,:],axis=0)
            
            eVeca=get_eVec(eVecsa)
            eVecb=get_eVec(eVecsb)

            def getRes(i_tf):
                tf=tfList[i_tf]
                # eVeca=eVecsa[tf,:,0]
                # eVecb=eVecsb[tf,:,0]
                nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                return np.real(preFactor*flip * (nom/denom-vevFactor*t[3]) )
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(opas[:1],opbs[:1],Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(tf,tErr/np.abs(tMean))
            # label = 'no GEVP' if i_tf==0 else '_nolegend_'
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],elinewidth=2,capsize=6,mfc='white',label=r'$t_{{sink}}={}$'.format(tf))

        axs[irow,0].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

        # icol=2
        # dat=getDat(opas,opbs,Gc,iso,projg,diags)
        # (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        # colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        # fmtDic={10:'s',12:'d',14:'o'}

        # for i_tf in range(len(tfList)):
        #     tf=tfList[i_tf]
        #     tMean=mean[i_tf];tErr=err[i_tf]
        #     label = 'GEVP' if i_tf==0 else '_nolegend_'
        #     # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
        #     #     fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
        #     axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
        #         fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white',label=label)
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim2)
        # axs[irow,0].legend()

        axs[irow,0].set_xlabel(r'$t_{ins}-t_{sink}/2$')
        axs[irow,0].set_ylabel('3pt/2pt')
        axs[irow,0].legend()

opas=[op00]; opbs=opas

diags=diags_jLoopless
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u}u+\bar{d}d,\quad p_c=(0,0,0)$")

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=4; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2.2,Lcol=5*2.2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        # fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+'; '+', '.join([op[0] for op in opas]) + '; '+', '.join([op[0] for op in opbs]) + (': '+title if title!=None else ''))
        # fig.suptitle(title)
    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[1],tRef)

            def get_eVec(eVecs):
                t=eVecs[:,0,0]
                t=eVecs[:,:,0]/t[:,None]
                return np.mean(t[tfAvgMin:tfAvgMax,:],axis=0)
            
            eVeca=get_eVec(eVecsa)
            eVecb=get_eVec(eVecsb)

            temp=np.array([[ (0 if j==i==1 else 1) for j in [0,1]]for i in [0,1]])

            def getRes(i_tf):
                tf=tfList[i_tf]
                # eVeca=eVecsa[tf,:,0]
                # eVecb=eVecsb[tf,:,0]

                temp0=np.array([[1,1],[1,0]]) if t[0][tf].shape==t[1][tf].shape==(2,2) else 1
                temp1=np.array([[1,1],[1,0]]) if t[0][tf].shape==t[1][tf].shape==(2,2) else 1

                nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
                denom=np.sqrt( np.real(np.conj(eVeca).T@(t[0][tf]*temp0)@eVeca) * np.real(np.conj(eVecb).T@(t[1][tf]*temp1)@eVecb)
                               *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1]
                  )
                
                return np.real(preFactor*flip * (nom/denom-vevFactor*t[3]) )
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(opas[:1],opbs[:1],Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print(tf,tErr/np.abs(tMean))
            label = 'no GEVP' if i_tf==0 else '_nolegend_'
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift-0.2,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white',label=label)

        axs[irow,0].set_xticks(range(-6,7,1))
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

        icol=2
        dat=getDat(opas,opbs,Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            label = 'GEVP' if i_tf==0 else '_nolegend_'
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift+0.2,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],label=label)
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim2)
        axs[irow,0].legend()
        axs[irow,0].set_xlabel(r'$t_{ins}-t_{sink}/2$')
        axs[irow,0].set_ylabel('3pt/2pt')
        
    return fig


diags=diags_jLoopless
# diags=diags_all

noD1ii=True

opas=[op00,op01]; opbs=[op00,op01]
# (Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
# (Gc,iso,projgs,preFactor)=('id','-',['P0'],1)
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
(Gc,iso,projgs,preFactor)=('gt','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
# (Gc,iso,projgs,preFactor)=('g5gz','+',['Pz'],-1j)
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
# (Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")
noD1ii=False

In [ ]:
diags=diags_jLoopless
# diags=diags_all

noD1ii=True

opas=[op00,op03]; opbs=[op00,op03]
(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1); print(Gc,iso,projgs,preFactor)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^4 u + \bar{d} \gamma^4 d,\quad p_c=(0,0,0)$")

(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j); print(Gc,iso,projgs,preFactor)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^5\gamma^3 u - \bar{d} \gamma^5\gamma^3 d,\quad p_c=(0,0,0)$")

(Gc,iso,projgs,preFactor)=('id','+',['P0'],1); print(Gc,iso,projgs,preFactor)
fig=run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} u + \bar{d} d,\quad p_c=(0,0,0)$")

with PdfPages('/p/project/pines/li47/code/forDownload/cA48_id+.pdf') as pdf:
    pdf.savefig(fig)

noD1ii=False

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=4; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2.2,Lcol=5*2.2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
        # fig.suptitle('(Gc,iso,projgs,preFactor)='+str((Gc,iso,projgsLabel,preFactorStr))+'; '+', '.join([op[0] for op in opas]) + '; '+', '.join([op[0] for op in opbs]) + (': '+title if title!=None else ''))
        # fig.suptitle(title)
    
    vevFactor = 1 if 'N_2pt-j' in diags and opas[0]==opbs[0] else 0

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[1],tRef)

            # [[] for j in [0,1]], for i in [0,1]]

            # print(t[0].shape)

            def get_eVec(eVecs):
                t=eVecs[:,0,0]
                t=eVecs[:,:,0]/t[:,None]
                return np.mean(t[tfAvgMin:tfAvgMax,:],axis=0)
            
            eVeca=get_eVec(eVecsa)
            eVecb=get_eVec(eVecsb)

            def getRes(i_tf):
                tf=tfList[i_tf]
                # eVeca=eVecsa[tf,:,0]
                # eVecb=eVecsb[tf,:,0]
                nom=np.conj(eVeca).T@t[2][i_tf]@eVecb
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                return np.real(preFactor*flip * (nom/denom-vevFactor*t[3]) )
                
            t=np.array([getRes(ind_tf) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(opas[:1],opbs[:1],Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            label = 'no GEVP' if i_tf==0 else '_nolegend_'
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift-0.2,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],mfc='white',label=label)

        axs[irow,0].set_xticks(range(-6,7,1))
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

        icol=2
        dat=getDat(opas,opbs,Gc,iso,projg,diags)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            label = 'GEVP' if i_tf==0 else '_nolegend_'
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift+0.2,tMean,tErr,\
                fmt=fmtDic[tf],color=['r','g','b'][i_tf],label=label)
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim2)
        axs[irow,0].legend()
        axs[irow,0].set_xlabel(r'$t_{ins}-t_{sink}/2$')
        axs[irow,0].set_ylabel('3pt/2pt')

    return fig

diags=diags_jLoopless

noD1ii=True

opas=[op00]; opbs=[op10,op13,op131]
(Gc,iso,projgs,preFactor)=('g5','-',['Pz'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^5 u -\bar{d} \gamma^5 d,\quad p_c=(0,0,1)$")

opas=[op00]; opbs=[op10,op13,op131]
(Gc,iso,projgs,preFactor)=('g5gt','-',['Pz'],1)
fig=run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title=r"$J=\bar{u} \gamma^5\gamma^4 u -\bar{d} \gamma^5\gamma^4 d,\quad p_c=(0,0,1)$")

with PdfPages('/p/project/pines/li47/code/forDownload/cA48_g5gt-.pdf') as pdf:
    pdf.savefig(fig)


noD1ii=False

In [ ]:
opas=[op00,op01]; opbs=opas
# opas=[op00]; opbs=[op10,op111]

diags=diags_jLoopless

test_NJNpi=0
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
opas=[op00,op01]; opbs=opas
# opas=[op00]; opbs=[op10,op111]

diags=diags_jLoopless

test_NJNpi=0
(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
opas=[op00,op01]; opbs=[op00,op01]
# opas=[op00,op01]; opbs=[op00]
opas=[op00]; opbs=[op10,op111]

diags=diags_jLoopless

test_NJNpi=1
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('id','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','+',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1

In [ ]:
opas=[op00]; opbs=[op00,op01]
# opas=[op00,op01]; opbs=[op00]
# opas=[op00]; opbs=[op10,op111]

diags=diags_jLoopless

test_NJNpi=1
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('id','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','+',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1

In [ ]:
opas=[op00]; opbs=[op00,op01]
opas=[op00,op01]; opbs=[op00]
# opas=[op00]; opbs=[op10,op111]

diags=diags_jLoopless

test_NJNpi=1
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('id','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('gt','-',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','+',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1

In [ ]:
opas=[op00]; opbs=[op10,op13,op131]

diags=diags_all
(Gc,iso,projgs,preFactor)=('gz','+',['Pz'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
diags=diags_jLoopless

noD1ii=True

opas=[op00,op03]; opbs=opas
(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

opas=[op00,op03]; opbs=opas
(Gc,iso,projgs,preFactor)=('g5gz','-',['Pz'],-1j)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

opas=[op00]; opbs=[op10,op13,op131]
(Gc,iso,projgs,preFactor)=('g5','-',['Pz'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

opas=[op00]; opbs=[op10,op13,op131]
(Gc,iso,projgs,preFactor)=('g5gt','-',['Pz'],1)
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)


noD1ii=False

In [ ]:
(opax,opay,opaz,opan)=(op00,op01,op02,op03)

# (opx,opy,opz,opn)=(op00,op01,op02,op03)
# (opx,opy,opz,opn)=(op10,op11,op12,op13)
(opx,opy,opz,opn)=(op10,op111,op121,op131)
# (opx,opy,opz,opn)=(op20,op21,op22,op23)
# (opx,opy,opz,opn)=(op30,op31,op32,op33)

(Gc,iso,projgs,preFactor)=('id','+',projgs_P012,1)
(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)

diags=diags_jLoopless
# diags=diags_all

opas=[opax]; opbs=[opx,opy]
test_NNpi=1; test_NpiNpi=1; test_NJNpi=-1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)


# opas=[opax]; opbs=[opx,opz]
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=0
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags,title='turn off NJNpi')
# test_NNpi=1; test_NpiNpi=1; test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

In [ ]:
diags=diags_all #- {'N_2pt-j&pi0i','N_2pt-j-pi0i'}
# diags=diags_jLoopless | {'N_2pt-j','N_2pt-j-pi0i'}
diags=diags_jLoopless #- {'B','W','Z'}

noD1ii=True

opas=[op00]; opbs=[op10,op111]
(Gc,iso,projgs,preFactor)=('id','+',['P01'],1)
test_NJNpi=-1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)
test_NJNpi=-1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

# (Gc,iso,projgs,preFactor)=('g5gt','-',['P01'],1)
# test_NJNpi=-1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
# test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

noD1ii=False

In [ ]:
diags=diags_all #- {'N_2pt-j&pi0i','N_2pt-j-pi0i'}
# diags=diags_jLoopless | {'N_2pt-j','N_2pt-j-pi0i'}
diags=diags_jLoopless #- {'B','W','Z'}
# diags=diags_jLoopless | diags_all

noD1ii=True

opas=[op00]; opbs=[op10,op13,op131,op132,op133]
opas=[op00]; opbs=[op10,op131]
print(opbs)

(Gc,iso,projgs,preFactor)=('g5','-',['Pz'],1)
# test_NJNpi=-1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
# test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

(Gc,iso,projgs,preFactor)=('g5gt','-',['Pz'],1)
# test_NJNpi=-1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
# test_NJNpi=1
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

# (Gc,iso,projgs,preFactor)=('g5gt','-',['P01'],-1j)
# test_NJNpi=-1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)
# test_NJNpi=1
# run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,diags)

noD1ii=False

In [ ]:
(Gc,iso,projgs,preFactor)=('id','-',['P01'],1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-2e-9,2e-9],xshift=-0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN','N_2pt-j'},axs=axs,color='b')

In [ ]:
(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-2e-9,2e-9],xshift=-0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN','N_2pt-j'},axs=axs,color='b')

(Gc,iso,projgs,preFactor)=('g5gt','-',['P01'],1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-2e-10,2e-10],xshift=-0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN','N_2pt-j'},axs=axs,color='b')

(Gc,iso,projgs,preFactor)=('g5gt','-',['P01'],-1j)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-2e-10,2e-10],xshift=-0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN','N_2pt-j'},axs=axs,color='b')

In [ ]:
(Gc,iso,projgs,preFactor)=('g5','-',['P01'],1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-9,1e-10],xshift=0.2)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j_50'},axs=axs,color='g',xshift=-0.1)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='b',xshift=0)


In [ ]:
projgs=['Pz']
(Gc,iso,projgs,preFactor)=('g5','-',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc+': u-d')
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-9,1e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

(Gc,iso,projgs,preFactor)=('g5','+',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc+': u+d')
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-5e-10,3e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

In [ ]:
projgs=['Pz']
(Gc,iso,projgs,preFactor)=('g5gz','-',projgs,-1j)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-7e-10,1e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

In [ ]:
projgs=['P0']

(Gc,iso,projgs,preFactor)=('id','-',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-10,1e-9],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

(Gc,iso,projgs,preFactor)=('gt','-',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-10,1e-9],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

projgs=['Pz']
(Gc,iso,projgs,preFactor)=('g5','-',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-9,1e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

(Gc,iso,projgs,preFactor)=('g5gz','-',projgs,-1j)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op00)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-10,2e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)

(Gc,iso,projgs,preFactor)=('g5gt','-',projgs,1)
fig,axs=yu.getFigAxs(1,1,8*1.5,10*1.5); fig.suptitle(Gc)
(opa,opb)=(op00,op10)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'NJN'},axs=axs,color='r',ylim1=[-1e-10,2e-10],xshift=0.3)
run3pt(opa,opb,Gc,iso,projgs,preFactor,{'N_2pt-j'},axs=axs,color='g',xshift=0)